# Advanced usage: The `MorphingWorkflow` classes: `MorphingWorkflowGlobal` and `MorphingWorkflowEurope` (v4 Update)

This notebook demonstrates the advanced, step-by-step approach to morphing EPW files using the specialized `MorphingWorkflow` classes (`MorphingWorkflowGlobal` and `MorphingWorkflowEurope`).

While the `morph_epw_*` functions are great for direct, one-shot tasks, the workflow classes are designed for complex projects where you need full control over **filename parsing, custom renaming, and process validation**. They enforce a safe, multi-step process that allows you to review and confirm each stage before executing the time-consuming morphing computation.

This notebook will focus on **`MorphingWorkflowGlobal`**, and will show a quick example using **`MorphingWorkflowEurope`** highlighting the differences.

**Note:** `pyfwg` now fully supports **FutureWeatherGenerator v4.0.x** (Global) and **v2.0.x** (Europe), which include a new keyword-based CLI, support for string-based parameter options, and additional output formats (like .met or .csv).

## The Three-Step Workflow

The class is designed to be used as a state machine, guiding you through a logical sequence:

1.  **`map_categories()`**: Analyze the input filenames to extract meaningful data (like city, building type, etc.).
2.  **`configure_and_preview()`**: Define all execution parameters, validate them, and generate a "dry run" plan of how the final files will be named and organized.
3.  **`execute_morphing()`**: Run the final computation, confident that your plan and configuration are correct.

## Using `MorphingWorkflowGlobal`

### General Setup

First, we'll import the necessary class and set up the paths for our files. 

**Important:** You must change the `jar_path` variable to the correct location of your Future Weather Generator `.jar` file. `pyfwg` will automatically detect the version and adapt the CLI command accordingly.

In [15]:
import os
import pandas as pd
from pyfwg import MorphingWorkflowGlobal, DEFAULT_GLOBAL_GCMS

# --- Configuration ---
# !!! IMPORTANT: Update this path to your V4 JAR !!!
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar"

# --- Define file paths for the examples ---
pattern_epw_dir = 'epws/w_pattern'
keyword_epw_dir = 'epws/wo_pattern'

### Step 1: Map Categories from Filenames

This is the first and most critical step for organizing your workflow. The `map_categories` method analyzes your source filenames and extracts meaningful data that will be used later for renaming the output files.

#### `map_categories` Parameters

The parameters you can use are:

*   `epw_files` (`List[str]`): **Required.** A list of paths to the EPW files you want to process.
*   `input_filename_pattern` (`Optional[str]`, default: `None`): A Python regex string with **named capture groups** (e.g., `(?P<city>...)`) to extract structured data from filenames.
*   `keyword_mapping` (`Optional[Dict]`, default: `None`): A dictionary of rules. It can be used to normalize values from the pattern or to search the entire filename for keywords to assign categories.

In [16]:
# Instantiate a workflow object for this example
workflow = MorphingWorkflowGlobal()

epw_files_without_pattern = [os.path.join(keyword_epw_dir, f) for f in os.listdir(keyword_epw_dir) if f.endswith('.epw')]

mapping_rules = {
    'city': {
        'Seville': ['sevilla', 'SVQ'],
        'London': ['london', 'gatwick']
    },
    'uhi': {
        'type-1': 'type-1',
        'type-2': 'type-2'
    }
}

workflow.map_categories(
    epw_files=epw_files_without_pattern,
    input_filename_pattern=None,
    keyword_mapping=mapping_rules
)

2025-12-30 12:39:49 - INFO - --- Step 1: Mapping categories from filenames ---
2025-12-30 12:39:49 - INFO - Mapped 'epws/wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw': {'city': 'London', 'uhi': 'type-2'}
2025-12-30 12:39:49 - INFO - Mapped 'epws/wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw': {'city': 'Seville', 'uhi': 'type-1'}
2025-12-30 12:39:49 - INFO - Category mapping complete.


### Step 2: Configure and Preview the Plan (V4 Update)

In V4, you can use descriptive strings for interpolation methods (e.g., `'AVG4P'`) and solar hour adjustments (e.g., `'By_Month'`). You can also specify the `fwg_version` to force the new CLI.

#### `configure_and_preview` Parameters

##### Workflow Parameters
*   `final_output_dir` (`str`): **Required.** The path for the final output files.
*   `output_filename_pattern` (`str`): **Required.** The template for final filenames.
*   `scenario_mapping` (`Optional[Dict]`, default: `None`): A dictionary to translate raw scenario names.
*   `fwg_jar_path` (`str`): **Required.** Path to the `.jar` file.
*   `fwg_version` (`Optional[str]`): Force a specific CLI version (e.g., `'4'`).
*   `fwg_output_type` (`str`, default: `'EPW'`): **New in V4/Europe v2.** Format of output files: `'EPW'`, `'SPAIN_MET'`, or `'PORTUGAL_CSV'`.

In [17]:
workflow.configure_and_preview(
    final_output_dir='./final_results_workflow_global',
    output_filename_pattern='{city}_{uhi}_{ssp}_{year}_interp-{fwg_interpolation_method_id}',
    scenario_mapping={'ssp245': 'SSP2-4.5', 'ssp585': 'SSP5-8.5'},
    
    # --- FWG Configuration ---
    fwg_jar_path=jar_path,
    fwg_gcms=['BCC_CSM2_MR'], 
    fwg_interpolation_method_id='AVG4P', # V4: Descriptive string
    fwg_epw_original_lcz=2,        
    fwg_target_uhi_lcz=3,          
    fwg_version='4',                # Force V4 syntax
    fwg_show_tool_output=True
)

2025-12-30 12:39:49 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---



          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global
  - EPWs to be Morphed (2 files):
    - GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    - sevilla_in_this_one_the_uhi_is_type-1.epw

  For input file: GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    -> Generated 'ssp126_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global\London_type-2_ssp126_2050_interp-AVG4P.epw
    -> Generated 'ssp245_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global\London_type-2_SSP2-4.5_2050_interp-AVG4P.epw
    -> Generated 'ssp370_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global\London_type-2_ssp370_2050_interp-AVG4P.epw
    -> Generated 'ssp585_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\

### Step 3: Execute the Morphing Process

This is the final step. The `execute_morphing` method runs the computation. With V4, it will use the new keyword-based CLI.

In [18]:
workflow.execute_morphing()

2025-12-30 12:39:49 - INFO - --- Step 4: Executing morphing workflow ---
2025-12-30 12:39:49 - INFO - Validating LCZ availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-12-30 12:39:49 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-12-30 12:39:49 - INFO - --- Applying UHI effect to GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw ---
2025-12-30 12:39:49 - INFO - Executing command: java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar" -u -epw=C:\Users\sanga\AppData\Local\Temp\tmp9ikq7m2w\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw -output_folder=C:\Users\sanga\AppData\Local\Temp\tmp9ikq7m2w\ -uhi=true:2:3 -output_type=EPW
2025-12-30 12:40:05 - INFO - UHI effect applied successfully.
2025-12-30 12:40:05 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-12-30 12:40:05 - INFO - Copied input file to temporary directory: ./morphing_temp_result


-------------------- Executing FWG for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw --------------------
  Full Command (for reference): java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar" -epw=D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw -models=BCC_CSM2_MR -ensemble=true -temp_shift_winter=0.0 -temp_shift_summer=0.0 -smooth_hours=72 -multithread=true -grid_interpolation_method=AVG4P -solar_correction=By_Month -diffuse_method=Engerer_2015 -uhi=true:2:3 -output_type=EPW
  --- FWG Real-time Output ---


2025-12-30 12:40:32 - INFO - Processing generated files in: ./morphing_temp_results\GBR_London.Gatwick.037760_IWEC_uhi_type-2
2025-12-30 12:40:32 - INFO - Skipping auxiliary file: '00_logs'
2025-12-30 12:40:32 - INFO - Skipping auxiliary file: '01_processing_transcripts'
2025-12-30 12:40:32 - INFO - Skipping auxiliary file: '02_morphing_errors_and_warnings'
2025-12-30 12:40:32 - INFO - Skipping auxiliary file: '03_model_variables'
2025-12-30 12:40:32 - INFO - Skipping auxiliary file: '04_epw_comparisons'
2025-12-30 12:40:32 - INFO - Moving './morphing_temp_results\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_-_LONDON-GATWICK_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2050.epw' to './final_results_workflow_global\London_type-2_ssp126_2050_interp-AVG4P.epw'
2025-12-30 12:40:32 - INFO - Moving './morphing_temp_results\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_-_LONDON-GATWICK_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2050.stat' to './final_results_workflow_global\London_ty

  --- End of FWG Output ---


2025-12-30 12:40:33 - WARNING - PermissionError deleting ./morphing_temp_results\GBR_London.Gatwick.037760_IWEC_uhi_type-2. Retrying in 0.5s... (Attempt 2/5)
2025-12-30 12:40:33 - INFO - Validating LCZ availability for sevilla_in_this_one_the_uhi_is_type-1.epw...
2025-12-30 12:40:33 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for sevilla_in_this_one_the_uhi_is_type-1.epw...
2025-12-30 12:40:33 - INFO - --- Applying UHI effect to sevilla_in_this_one_the_uhi_is_type-1.epw ---
2025-12-30 12:40:33 - INFO - Executing command: java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar" -u -epw=C:\Users\sanga\AppData\Local\Temp\tmpv9l3_d4y\sevilla_in_this_one_the_uhi_is_type-1.epw -output_folder=C:\Users\sanga\AppData\Local\Temp\tmpv9l3_d4y\ -uhi=true:2:3 -output_type=EPW
2025-12-30 12:40:48 - INFO - UHI effect applied successfully.
2025-12-30 12:40:48 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-12-30 12:40:48 - IN


-------------------- Executing FWG for sevilla_in_this_one_the_uhi_is_type-1.epw --------------------
  Full Command (for reference): java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v4.0.2.jar" -epw=D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1\sevilla_in_this_one_the_uhi_is_type-1.epw D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw -models=BCC_CSM2_MR -ensemble=true -temp_shift_winter=0.0 -temp_shift_summer=0.0 -smooth_hours=72 -multithread=true -grid_interpolation_method=AVG4P -solar_correction=By_Month -diffuse_method=Engerer_2015 -uhi=true:2:3 -output_type=EPW
  --- FWG Real-time Output ---


2025-12-30 12:41:14 - INFO - Processing generated files in: ./morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1
2025-12-30 12:41:14 - INFO - Skipping auxiliary file: '00_logs'
2025-12-30 12:41:14 - INFO - Skipping auxiliary file: '01_processing_transcripts'
2025-12-30 12:41:14 - INFO - Skipping auxiliary file: '02_morphing_errors_and_warnings'
2025-12-30 12:41:14 - INFO - Skipping auxiliary file: '03_model_variables'
2025-12-30 12:41:14 - INFO - Skipping auxiliary file: '04_epw_comparisons'
2025-12-30 12:41:14 - INFO - Moving './morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2050.epw' to './final_results_workflow_global\Seville_type-1_ssp126_2050_interp-AVG4P.epw'
2025-12-30 12:41:14 - INFO - Moving './morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_ssp126_2050.stat' to './final_results_workflow_global\Seville_type-1_ssp126_2050_interp-

  --- End of FWG Output ---


2025-12-30 12:41:15 - INFO - Skipping auxiliary file: 'UHI_AirTemperature_10000'
2025-12-30 12:41:15 - INFO - Deleting temporary directory: ./morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1
2025-12-30 12:41:15 - WARNING - PermissionError deleting ./morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1. Retrying in 0.5s... (Attempt 1/5)
2025-12-30 12:41:15 - INFO - Morphing workflow finished.


Let's take a look at the files we just generated:

In [19]:
output_files = [i for i in os.listdir('./final_results_workflow_global')]
output_files

['London_type-2_ssp126_2050_interp-AVG4P.epw',
 'London_type-2_ssp126_2050_interp-AVG4P.stat',
 'London_type-2_ssp126_2080_interp-AVG4P.epw',
 'London_type-2_ssp126_2080_interp-AVG4P.stat',
 'London_type-2_SSP2-4.5_2050_interp-AVG4P.epw',
 'London_type-2_SSP2-4.5_2050_interp-AVG4P.stat',
 'London_type-2_SSP2-4.5_2080_interp-AVG4P.epw',
 'London_type-2_SSP2-4.5_2080_interp-AVG4P.stat',
 'London_type-2_ssp370_2050_interp-AVG4P.epw',
 'London_type-2_ssp370_2050_interp-AVG4P.stat',
 'London_type-2_ssp370_2080_interp-AVG4P.epw',
 'London_type-2_ssp370_2080_interp-AVG4P.stat',
 'London_type-2_SSP5-8.5_2050_interp-AVG4P.epw',
 'London_type-2_SSP5-8.5_2050_interp-AVG4P.stat',
 'London_type-2_SSP5-8.5_2080_interp-AVG4P.epw',
 'London_type-2_SSP5-8.5_2080_interp-AVG4P.stat',
 'Seville_type-1_ssp126_2050_interp-AVG4P.epw',
 'Seville_type-1_ssp126_2050_interp-AVG4P.stat',
 'Seville_type-1_ssp126_2080_interp-AVG4P.epw',
 'Seville_type-1_ssp126_2080_interp-AVG4P.stat',
 'Seville_type-1_SSP2-4.5_2050

## Using `MorphingWorkflowEurope`

### Europe v2 Example

**FutureWeatherGenerator Europe v2** also adopts the keyword-based CLI. Remember to use `fwg_rcm_pairs` instead of `fwg_gcms`, and `{rcp}` instead of `{ssp}` for renaming.

In [20]:
from pyfwg import MorphingWorkflowEurope, DEFAULT_EUROPE_RCMS

# !!! IMPORTANT: Update this path to your Europe v2 JAR !!!
jar_path_europe = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar"

workflow_eu = MorphingWorkflowEurope()
workflow_eu.map_categories(
    epw_files=epw_files_without_pattern,
    keyword_mapping=mapping_rules
)

workflow_eu.configure_and_preview(
    delete_temp_files=False,
    final_output_dir='./final_results_workflow_europe',
    output_filename_pattern='{city}_{uhi}_{rcp}_{year}',
    scenario_mapping={'rcp26': 'RCP-2.6'},
    fwg_jar_path=jar_path_europe,
    fwg_rcm_pairs=[list(DEFAULT_EUROPE_RCMS)[0]],
    fwg_epw_original_lcz=2,
    fwg_target_uhi_lcz=3,
    fwg_version='2', # Europe v2 keyword CLI
    fwg_output_type='PORTUGAL_CSV' # New: Generate CSV files
)

workflow_eu.execute_morphing()

2025-12-30 12:41:15 - INFO - --- Step 1: Mapping categories from filenames ---
2025-12-30 12:41:15 - INFO - Mapped 'epws/wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw': {'city': 'London', 'uhi': 'type-2'}
2025-12-30 12:41:15 - INFO - Mapped 'epws/wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw': {'city': 'Seville', 'uhi': 'type-1'}
2025-12-30 12:41:15 - INFO - Category mapping complete.
2025-12-30 12:41:15 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---
2025-12-30 12:41:15 - INFO - --- Step 4: Executing morphing workflow ---
2025-12-30 12:41:15 - INFO - Validating LCZ availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-12-30 12:41:15 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-12-30 12:41:15 - INFO - --- Applying UHI effect to GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw ---
2025-12-30 12:41:15 - INFO - Executing command: java -jar "D:\OneDrive - Universi


          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe
  - EPWs to be Morphed (2 files):
    - GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    - sevilla_in_this_one_the_uhi_is_type-1.epw

  For input file: GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    -> Generated 'rcp26_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe\London_type-2_RCP-2.6_2050.epw
    -> Generated 'rcp45_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe\London_type-2_rcp45_2050.epw
    -> Generated 'rcp85_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe\London_type-2_rcp85_2050.epw
    -> Generated 'rcp26_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe\London_ty

2025-12-30 12:41:32 - INFO - UHI effect applied successfully.
2025-12-30 12:41:32 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-12-30 12:41:32 - INFO - Copied input file to temporary directory: ./morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw



-------------------- Executing FWG for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw --------------------
  Full Command (for reference): java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar" -epw=D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw -models=MOHC_HadGEM2_ES_SMHI_RCA4 -ensemble=true -temp_shift_winter=0.0 -temp_shift_summer=0.0 -smooth_hours=72 -multithread=true -grid_interpolation_method=IDW -solar_correction=By_Month -diffuse_method=Engerer_2015 -uhi=true:2:3 -output_type=PORTUGAL_CSV


2025-12-30 12:42:11 - INFO - Processing generated files in: ./morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2
2025-12-30 12:42:11 - INFO - Skipping auxiliary file: '00_logs'
2025-12-30 12:42:11 - INFO - Skipping auxiliary file: '01_processing_transcripts'
2025-12-30 12:42:11 - INFO - Skipping auxiliary file: '02_morphing_errors_and_warnings'
2025-12-30 12:42:11 - INFO - Skipping auxiliary file: '03_model_variables'
2025-12-30 12:42:11 - INFO - Skipping auxiliary file: '04_epw_comparisons'
2025-12-30 12:42:11 - INFO - Moving './morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_-_LONDON-GATWICK_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp26_2050.csv' to './final_results_workflow_europe\London_type-2_RCP-2.6_2050.csv'
2025-12-30 12:42:11 - INFO - Moving './morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_-_LONDON-GATWICK_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp26_2050.stat' to './final_results_workflow_europe\Lo


-------------------- Executing FWG for sevilla_in_this_one_the_uhi_is_type-1.epw --------------------
  Full Command (for reference): java -jar "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v2.0.2.jar" -epw=D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1\sevilla_in_this_one_the_uhi_is_type-1.epw D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw -models=MOHC_HadGEM2_ES_SMHI_RCA4 -ensemble=true -temp_shift_winter=0.0 -temp_shift_summer=0.0 -smooth_hours=72 -multithread=true -grid_interpolation_method=IDW -solar_correction=By_Month -diffuse_method=Engerer_2015 -uhi=true:2:3 -output_type=PORTUGAL_CSV


2025-12-30 12:43:06 - INFO - Processing generated files in: ./morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1
2025-12-30 12:43:06 - INFO - Skipping auxiliary file: '00_logs'
2025-12-30 12:43:06 - INFO - Skipping auxiliary file: '01_processing_transcripts'
2025-12-30 12:43:06 - INFO - Skipping auxiliary file: '02_morphing_errors_and_warnings'
2025-12-30 12:43:06 - INFO - Skipping auxiliary file: '03_model_variables'
2025-12-30 12:43:06 - INFO - Skipping auxiliary file: '04_epw_comparisons'
2025-12-30 12:43:06 - INFO - Moving './morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp26_2050.csv' to './final_results_workflow_europe\Seville_type-1_RCP-2.6_2050.csv'
2025-12-30 12:43:06 - INFO - Moving './morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1\ESP_-_SEVILLA_Ensemble_UHI_Target_LCZ-3_Original_LCZ-2_rcp26_2050.stat' to './final_results_workflow_europe\Seville_type-1_RCP-2.6_2050

Let's take a look at the files we just generated:

In [21]:
output_files = [i for i in os.listdir('./final_results_workflow_europe')]
output_files

['London_type-2_RCP-2.6_2050.csv',
 'London_type-2_RCP-2.6_2050.stat',
 'London_type-2_RCP-2.6_2080.csv',
 'London_type-2_RCP-2.6_2080.stat',
 'London_type-2_rcp45_2050.csv',
 'London_type-2_rcp45_2050.stat',
 'London_type-2_rcp45_2080.csv',
 'London_type-2_rcp45_2080.stat',
 'London_type-2_rcp85_2050.csv',
 'London_type-2_rcp85_2050.stat',
 'London_type-2_rcp85_2080.csv',
 'London_type-2_rcp85_2080.stat',
 'Seville_type-1_RCP-2.6_2050.csv',
 'Seville_type-1_RCP-2.6_2050.stat',
 'Seville_type-1_RCP-2.6_2080.csv',
 'Seville_type-1_RCP-2.6_2080.stat',
 'Seville_type-1_rcp45_2050.csv',
 'Seville_type-1_rcp45_2050.stat',
 'Seville_type-1_rcp45_2080.csv',
 'Seville_type-1_rcp45_2080.stat',
 'Seville_type-1_rcp85_2050.csv',
 'Seville_type-1_rcp85_2050.stat',
 'Seville_type-1_rcp85_2080.csv',
 'Seville_type-1_rcp85_2080.stat']

## Cleaning up

Once the workflow is finished, you can delete the temporary folders created during the process if you don't need them anymore. If you did not enable `delete_temp_files=True` in `configure_and_preview`, you can do it manually.

**Note on Analysis Files:** In FutureWeatherGenerator v4 (Global) and v2 (Europe), the tool generates several analysis folders (e.g., `00_logs`, `01_processing_transcripts`, etc.) when enabled. These folders are **not** moved to the final output directory; they remain in the temporary directory for inspection. If you delete the temporary directory, these files will be lost.

In [22]:
import shutil
import os

# Clean up the results folders
for folder in ['./morphing_temp_results', './morphing_temp_results_europe', './final_results_workflow_global', './final_results_workflow_europe', './00_logs']:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Deleted: {folder}")

Deleted: ./morphing_temp_results
Deleted: ./morphing_temp_results_europe
Deleted: ./final_results_workflow_global
Deleted: ./final_results_workflow_europe
